# Visual Analytics Project 1

**Theme:** Charting with Code — Energy, AI, and Climate
**Submitted By:** Chirasmayee B

## Table of Contents
1. Data & Setup
2. Global Electricity & Mix
3. AI Energy Scenarios
4. Data Center Types (Capacity, PUE, Load)
5. Regional Views
6. Per-Capita CO₂ Choropleth (Animated)


**Data Provenance:** IEA *Energy & AI* workbook; Our World in Data (OWID) electricity & CO₂ datasets.


In [ ]:
pip install pandas
pip install plotly
pip install openpyxl
pip install numpy


## Line chart

<!-- REASONING_BLOCK -->
### Why a line chart?
- **Purpose:** Show how a metric changes **over time**.
- **Audience task:** Compare **trends and inflection points** across groups (e.g., regions or sources).
- **How to read:** Left→right = earlier→later; steeper slope = faster change.
- **Caveats:** With **sparse years**, lines can imply continuity that doesn't exist. Consider bars or dots if years are far apart.


In [ ]:
import pandas as pd
import plotly.express as px

# Load OWID energy data
#url = "https://raw.githubusercontent.com/owid/energy-data/master/owid-energy-data.csv"
df = pd.read_csv("owid-energy-data.csv")

# Keep the 'World' aggregate
world = df[df["country"] == "World"].copy()

# Select a few sources that exist in your columns
cols = [
    "coal_electricity",
    "gas_electricity",
    "nuclear_electricity",
    "renewables_electricity",  # (this includes wind/solar/hydro/other)
]
data = world[["year"] + cols].melt(
    id_vars="year", var_name="Source", value_name="Electricity (TWh)"
)

# Clean display names
data["Source"] = (data["Source"]
                  .str.replace("_electricity", "", regex=False)
                  .str.title())

# Drop rows where value is missing
data = data.dropna(subset=["Electricity (TWh)"])

fig = px.line(
    data, x="year", y="Electricity (TWh)", color="Source",
    title="Global Electricity Generation by Source (OWID)"
)
fig.update_layout(xaxis_title="Year", yaxis_title="TWh")
fig.show()

#fig.write_html("line_chart_energy_mix.html")


## Bar chart

<!-- REASONING_BLOCK -->
### Why a bar chart?
- **Purpose:** Compare **discrete categories** at a **single point** (or a few points) in time.
- **Audience task:** Rank categories; read exact values via labels/hover.
- **How to read:** Taller bar = larger value; grouped bars compare scenarios/years side by side.
- **Caveats:** Too many groups can clutter—prefer **small multiples** if needed.


In [ ]:
import pandas as pd
import plotly.express as px

#url = "https://raw.githubusercontent.com/owid/energy-data/master/owid-energy-data.csv"
df = pd.read_csv("owid-energy-data.csv")
world = df[df["country"] == "World"].copy()

sources = [
    "coal_electricity", "gas_electricity", "nuclear_electricity",
    "hydro_electricity", "wind_electricity", "solar_electricity",
    "other_renewable_electricity"
]
year = 2022  # pick the year you want
row = world[world["year"] == year][sources].T.reset_index()
row.columns = ["Source", "Electricity (TWh)"]

row["Source"] = (row["Source"]
                 .str.replace("_electricity", "", regex=False)
                 .str.replace("_", " ")
                 .str.title())

fig = px.bar(row, x="Source", y="Electricity (TWh)",
             title=f"Global Electricity by Source in {year} (OWID)")
fig.update_layout(xaxis_tickangle=-30)
fig.show()

#fig.write_html("bar_chart_energy_mix.html")


## Stacked area chart

<!-- REASONING_BLOCK -->
### Why a stacked area chart?
- **Purpose:** Show **parts of a whole over time** (composition + total).
- **Audience task:** See whether the **mix** (e.g., renewables vs fossil) is shifting.
- **How to read:** The vertical sum is the total; each colored band is a component.
- **Caveats:** Hard to read small categories and exact values; keep categories few.


In [ ]:
import pandas as pd
import plotly.express as px

#url = "https://raw.githubusercontent.com/owid/energy-data/master/owid-energy-data.csv"
df = pd.read_csv("owid-energy-data.csv")
world = df[df["country"] == "World"].copy()

# Choose several sources; all exist per your column list
sources = [
    "coal_electricity",
    "gas_electricity",
    "nuclear_electricity",
    "hydro_electricity",
    "wind_electricity",
    "solar_electricity",
    "other_renewable_electricity",
]
data = world[["year"] + sources].melt(
    id_vars="year", var_name="Source", value_name="Electricity (TWh)"
)
data["Source"] = (data["Source"]
                  .str.replace("_electricity", "", regex=False)
                  .str.replace("_", " ")
                  .str.title())
data = data.dropna(subset=["Electricity (TWh)"])

fig = px.area(
    data, x="year", y="Electricity (TWh)", color="Source",
    title="Global Electricity Mix by Source (Stacked Area, OWID)"
)
fig.update_layout(xaxis_title="Year", yaxis_title="TWh")
fig.show()

#fig.write_html("stacked_area_energy_mix.html")

## Choropleth map

<!-- REASONING_BLOCK -->
### Why a choropleth map?
- **Purpose:** Compare a metric **across geography** at a glance.
- **Audience task:** Spot **regional hotspots** and patterns.
- **How to read:** Darker color = higher value; hover shows exact values.
- **Caveats:** Maps emphasize **area**, not population; ensure country names/ISO codes map correctly.


In [ ]:
import pandas as pd
import plotly.express as px

# 1) Load your CSV (adjust the path if needed)
csv_path = "co2-emissions-per-capita.csv"
df = pd.read_csv(csv_path)

# 2) Standardize column names used below (change these if your headers differ)
# Expected columns: "Entity", "Year", "Annual CO₂ emissions (per capita)"
# If your file uses a different CO2 column name, print df.columns and update the line below.
df = df.rename(columns={
    "Entity": "Country",
    "Annual CO₂ emissions (per capita)": "CO2_per_capita"
})

# 3) Pick the year to plot; fallback to the latest available if missing
year_to_plot = 2024
years_available = sorted(df["Year"].dropna().unique())
if year_to_plot not in years_available:
    year_to_plot = max(years_available)

# 4) Filter & clean
plot_df = (
    df[df["Year"] == year_to_plot]
    .dropna(subset=["Country", "CO2_per_capita"])
    .copy()
)

# Optional: if your CSV contains aggregates like "World", drop them (maps only support countries)
exclude = {"World", "Asia", "Europe", "Africa", "North America", "South America", "Oceania"}
plot_df = plot_df[~plot_df["Country"].isin(exclude)]

# 5) Build the choropleth
fig = px.choropleth(
    plot_df,
    locations="Country",
    locationmode="country names",   # uses country names directly
    color="CO2_per_capita",
    color_continuous_scale="Reds",
    hover_name="Country",
    hover_data={"CO2_per_capita": ":.2f", "Country": False},
    title=f"Per-Capita CO₂ Emissions by Country — {year_to_plot}",
)

# Optional styling
fig.update_layout(
    coloraxis_colorbar=dict(title="tCO₂ per person"),
    margin=dict(l=10, r=10, t=60, b=10)
)

fig.show()


#### Adding time slider animation

In [ ]:
import pandas as pd
import plotly.express as px

# 1) Load CSV
csv_path = "co2-emissions-per-capita.csv"  # adjust if needed
df = pd.read_csv(csv_path)

# 2) Standardize column names (update if yours differ)
df = df.rename(columns={
    "Entity": "Country",
    "Annual CO₂ emissions (per capita)": "CO2_per_capita"
})

# 3) Clean types and rows
# Ensure Year is numeric and drop rows without year or value
df["Year"] = pd.to_numeric(df["Year"], errors="coerce")
df = df.dropna(subset=["Year", "Country", "CO2_per_capita"])
df["Year"] = df["Year"].astype(int)

# Optional: drop aggregates that aren't countries (keeps the map clean)
exclude = {"World", "Asia", "Europe", "Africa", "North America", "South America", "Oceania"}
df = df[~df["Country"].isin(exclude)]

# 4) Build an explicit, sorted order for the animation slider
years_sorted = sorted(df["Year"].unique().tolist())

# 5) Draw animated choropleth with forced year order
fig = px.choropleth(
    df.sort_values(["Year", "Country"]),
    locations="Country",
    locationmode="country names",
    color="CO2_per_capita",
    hover_name="Country",
    animation_frame="Year",
    category_orders={"Year": years_sorted},   # <-- force sequential order
    color_continuous_scale="Reds",
    title="Per-Capita CO₂ Emissions by Country Over Time"
)

# Optional styling
fig.update_layout(
    coloraxis_colorbar=dict(title="tCO₂ per person"),
    margin=dict(l=10, r=10, t=60, b=10)
)

fig.show()


#### Data Cleaning: Removing North America excl. USA, Asia excl. China, India

In [ ]:
import pandas as pd
import plotly.express as px

# 1) Load CSV
csv_path = "co2-emissions-per-capita.csv"  # adjust path if needed
df = pd.read_csv(csv_path)

# 2) Standardize column names (update if your headers differ)
# Expected columns in many OWID exports: "Entity", "Code", "Year", "Annual CO₂ emissions (per capita)"
rename_map = {}
if "Entity" in df.columns: rename_map["Entity"] = "Country"
if "Code" in df.columns: rename_map["Code"] = "ISO3"
if "Annual CO₂ emissions (per capita)" in df.columns:
    rename_map["Annual CO₂ emissions (per capita)"] = "CO2_per_capita"
df = df.rename(columns=rename_map)

# 3) Inspect how US and China appear in your file
print("Sample rows for US-like names:")
print(df[df["Country"].str.contains("United|USA|America", case=False, na=False)].head(10))
print("\nSample rows for China-like names:")
print(df[df["Country"].str.contains("China", case=False, na=False)].head(10))

# 4) Ensure numeric year and drop incomplete rows
df["Year"] = pd.to_numeric(df["Year"], errors="coerce")
df = df.dropna(subset=["Year", "Country", "CO2_per_capita"])
df["Year"] = df["Year"].astype(int)

# 5) (Optional) drop aggregates that aren't countries
exclude = {"World", "Asia", "Asia (excl. China and India)", "Europe", "Africa", "North America", "North America (excl. USA)", "South America", "Oceania"}
df = df[~df["Country"].isin(exclude)]

# 6) Build an explicit year order for a proper time slider
years_sorted = sorted(df["Year"].unique().tolist())

# 7A) Preferred: use ISO3 codes if available (most reliable mapping)
if "ISO3" in df.columns and df["ISO3"].notna().any():
    data_for_map = df.sort_values(["Year", "ISO3"])
    fig = px.choropleth(
        data_for_map,
        locations="ISO3",
        color="CO2_per_capita",
        hover_name="Country",
        animation_frame="Year",
        category_orders={"Year": years_sorted},
        color_continuous_scale="Reds",
        title="Per-Capita CO₂ Emissions by Country Over Time"
    )
else:
    # 7B) Fallback: use country names (works if names match Plotly’s)
    data_for_map = df.sort_values(["Year", "Country"])
    fig = px.choropleth(
        data_for_map,
        locations="Country",
        locationmode="country names",
        color="CO2_per_capita",
        hover_name="Country",
        animation_frame="Year",
        category_orders={"Year": years_sorted},
        color_continuous_scale="Reds",
        title="Per-Capita CO₂ Emissions by Country Over Time"
    )

# 8) Style
fig.update_layout(
    coloraxis_colorbar=dict(title="tCO₂ per person"),
    margin=dict(l=10, r=10, t=60, b=10)
)

fig.show()

#fig.write_html("co2_emissions_choropleth.html")

## Bar chart

<!-- REASONING_BLOCK -->
### Why a bar chart?
- **Purpose:** Compare **discrete categories** at a **single point** (or a few points) in time.
- **Audience task:** Rank categories; read exact values via labels/hover.
- **How to read:** Taller bar = larger value; grouped bars compare scenarios/years side by side.
- **Caveats:** Too many groups can clutter—prefer **small multiples** if needed.


In [ ]:
import pandas as pd
import plotly.express as px

# Load Excel
file_path = "Data_annex_Energy_and_AI.xlsx"
world_df = pd.read_excel(file_path, sheet_name="World Data", header=None)

# Row 23 = "Electricity consumption (TWh)" → Total
consumption_row = world_df.iloc[23]

# Build tidy dataset
data = []
# Observed years
data += [("Observed", 2020, consumption_row[3])]
data += [("Observed", 2023, consumption_row[4])]
data += [("Observed", 2024, consumption_row[5])]

# Scenario projections
data += [("Base", 2030, consumption_row[7]), ("Base", 2035, consumption_row[8])]
data += [("Lift-Off", 2030, consumption_row[10]), ("Lift-Off", 2035, consumption_row[11])]
data += [("High Efficiency", 2030, consumption_row[13]), ("High Efficiency", 2035, consumption_row[14])]
data += [("Headwinds", 2030, consumption_row[16]), ("Headwinds", 2035, consumption_row[17])]

df = pd.DataFrame(data, columns=["Scenario", "Year", "Electricity (TWh)"])

# Rename scenarios for legend clarity
scenario_map = {
    "Observed": "Observed (historical)",
    "Base": "Base – expected trajectory",
    "Lift-Off": "Lift-Off – rapid adoption",
    "High Efficiency": "High Efficiency – tech breakthroughs",
    "Headwinds": "Headwinds – slower growth"
}
df["Scenario"] = df["Scenario"].map(scenario_map)

# Plot grouped bar chart
fig = px.bar(
    df,
    x="Year",
    y="Electricity (TWh)",
    color="Scenario",
    barmode="group",
    text="Electricity (TWh)",
    title="AI Data Center Electricity Consumption: Observed vs Scenario Futures"
)

fig.update_traces(textposition="outside")
fig.update_layout(
    xaxis=dict(type="category"),
    yaxis_title="Electricity (TWh)"
)
fig.update_layout(width=1200, height=700)
fig.show()

#fig.write_html("iea_energy_ai_grouped_bar_chart.html")


## Scatter Plot

In [ ]:
import pandas as pd
import plotly.express as px

# Load Excel
file_path = "Data_annex_Energy_and_AI.xlsx"
wd = pd.read_excel(file_path, sheet_name="World Data", header=None)

col_2024 = 5  # column index for 2024

# Row indices for attributes
rows_installed = {"Hyperscale": 5, "Colocation & Service": 6, "Enterprise": 7}
rows_pue       = {"Hyperscale": 14, "Colocation & Service": 15, "Enterprise": 16}
rows_load      = {"Hyperscale": 19, "Colocation & Service": 20, "Enterprise": 21}

# Build dataset
records = []
for cat in rows_installed:
    records.append({
        "Category": cat,
        "Installed Capacity (GW)": float(wd.iat[rows_installed[cat], col_2024]),
        "PUE": float(wd.iat[rows_pue[cat], col_2024]),
        "Load Factor (%)": float(wd.iat[rows_load[cat], col_2024])
    })

df = pd.DataFrame(records)

# Scatter/bubble plot
fig = px.scatter(
    df,
    x="Installed Capacity (GW)",
    y="Load Factor (%)",
    size="PUE",
    color="Category",
    hover_data=["PUE"],
    title="Data Center Types Compared (2024): Capacity vs Load vs Efficiency"
)

fig.show()


#### Scatter Plot with Bubble representing the PUE

In [ ]:
import pandas as pd
import plotly.express as px

# --- Load from your Excel ---
file_path = "Data_annex_Energy_and_AI.xlsx"   # adjust path if needed
wd = pd.read_excel(file_path, sheet_name="World Data", header=None)

col_2024 = 5  # column index for 2024 in this sheet layout

# Row indices (from the sheet) for each attribute & category
rows_installed = {"Hyperscale": 5, "Colocation & Service": 6, "Enterprise": 7}
rows_pue       = {"Hyperscale": 14, "Colocation & Service": 15, "Enterprise": 16}
rows_load      = {"Hyperscale": 19, "Colocation & Service": 20, "Enterprise": 21}

# Build tidy 2024 dataframe with REAL values
records = []
for cat in rows_installed:
    records.append({
        "Category": cat,
        "Installed Capacity (GW)": float(wd.iat[rows_installed[cat], col_2024]),
        "PUE": float(wd.iat[rows_pue[cat], col_2024]),
        "Load Factor (%)": float(wd.iat[rows_load[cat], col_2024])
    })
df = pd.DataFrame(records)

# ============== 1) Bubble scatter (PUE visible as size + text) ==============
fig = px.scatter(
    df,
    x="Installed Capacity (GW)",
    y="Load Factor (%)",
    color="Category",
    size="PUE",                 # smaller bubble = better efficiency (lower PUE)
    size_max=60,
    hover_data={"PUE":":.2f"},
    text=df["PUE"].round(2),    # print PUE on each point
    title="2024: Capacity vs Load (Bubble size/text = PUE; smaller = more efficient)"
)
fig.update_traces(textposition="top center")
fig.update_layout(
    xaxis_title="Installed Capacity (GW)",
    yaxis_title="Load Factor (%)",
    legend_title="Category"
)
fig.show()
fig.write_html("iea_energy_ai_bubble_chart.html")
# ===== Optional: invert size so bigger = better (Efficiency = 1/PUE) =====
'''
df["Efficiency (1/PUE)"] = 1 / df["PUE"]
fig_eff = px.scatter(
    df,
    x="Installed Capacity (GW)",
    y="Load Factor (%)",
    color="Category",
    size="Efficiency (1/PUE)",  # now larger bubble means better efficiency
    size_max=60,
    hover_data={"PUE":":.2f", "Efficiency (1/PUE)":":.2f"},
    text=df["PUE"].round(2),
    title="2024: Capacity vs Load (Bubble size = 1/PUE; larger = more efficient)"
)
fig_eff.update_traces(textposition="top center")
fig_eff.update_layout(
    xaxis_title="Installed Capacity (GW)",
    yaxis_title="Load Factor (%)",
    legend_title="Category"
)
fig_eff.show()
'''

# ============== 2) Scatter Matrix (SPLOM) for all three attributes ==============
'''
fig2 = px.scatter_matrix(
    df,
    dimensions=["Installed Capacity (GW)", "Load Factor (%)", "PUE"],
    color="Category",
    symbol="Category",
    title="2024: Pairwise Comparison — Capacity, Load, and PUE"
)
fig2.update_traces(diagonal_visible=False)
fig2.update_layout(legend_title="Category")
fig2.show()
'''
# Quick print of the values used (sanity check)
print(df)


#### Line Chart for Installed Capacity(GW)

In [ ]:
import pandas as pd
import plotly.express as px

# Load Excel
file_path = "Data_annex_Energy_and_AI.xlsx"
regional_df = pd.read_excel(file_path, sheet_name="Regional Data", header=None)

# Extract installed capacity rows (World to China)
rows_capacity = list(range(5, 14))

# Prepare data
records = []
for r in rows_capacity:
    region = regional_df.iat[r, 1]
    for year_label, c in [(2020, 2), (2023, 3), (2024,4),(2030, 6)]:
        val = regional_df.iat[r, c]
        if pd.notna(val):
            records.append({
                "Region": region,
                "Year": str(year_label),
                "Capacity (GW)": float(val)
            })

df_capacity = pd.DataFrame(records)

# Create line chart
fig = px.line(
    df_capacity,
    x="Year",
    y="Capacity (GW)",
    color="Region",
    markers=True,
    title="Installed Capacity (GW) by Region (2020, 2023, 2030 Base Case)"
)
fig.update_traces(mode="lines+markers")
fig.update_layout(
    yaxis_title="Installed Capacity (GW)",
    xaxis_title="Year",
    legend_title="Region"
)

fig.show()


#### Line Chart for Electricity consumption (TWh) without North America and Asia Pacific

In [ ]:
import pandas as pd
import plotly.express as px

# Load Excel
file_path = "Data_annex_Energy_and_AI.xlsx"
regional_df = pd.read_excel(file_path, sheet_name="Regional Data", header=None)

# Extract electricity consumption rows (based on your file structure: rows 56–64)
rows_capacity = list(range(56, 65))

# Prepare data
records = []
for r in rows_capacity:
    region = regional_df.iat[r, 1]
    for year_label, c in [(2020, 2), (2023, 3), (2024, 4), (2030, 6)]:
        val = regional_df.iat[r, c]
        if pd.notna(val):
            records.append({
                "Region": region,
                "Year": str(year_label),
                "Electricity consumption (TWh)": float(val)
            })
print(regional_df.iloc[56:65, 0:10])

df_capacity = pd.DataFrame(records)

# Filter out North America & Asia Pacific
df_capacity = df_capacity[~df_capacity["Region"].isin(["North America", "Asia Pacific"])]

# Create line chart
fig = px.line(
    df_capacity,
    x="Year",
    y="Electricity consumption (TWh)",
    color="Region",
    markers=True,
    title="Electricity consumption (TWh) by Region (Excluding North America & Asia Pacific)"
)
fig.update_traces(mode="lines+markers")
fig.update_layout(
    yaxis_title="Electricity consumption (TWh)",
    xaxis_title="Year",
    legend_title="Region"
)

fig.show()
#fig.write_html("iea_energy_ai_line_chart.html")


## Dumbbell Chart

In [ ]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# --- Load from your Excel ---
file_path = "Data_annex_Energy_and_AI.xlsx"   # adjust path if needed
wd = pd.read_excel(file_path, sheet_name="World Data", header=None)

# Column indices for the sheet layout we inspected earlier
COL_2024 = 5          # observed 2024
COL_BASE_2030 = 7     # Base 2030 projection

# Row indices (from the sheet) for each attribute & category
rows_installed = {"Hyperscale": 5, "Colocation & Service": 6, "Enterprise": 7}
rows_pue       = {"Hyperscale": 14, "Colocation & Service": 15, "Enterprise": 16}
rows_load      = {"Hyperscale": 19, "Colocation & Service": 20, "Enterprise": 21}

# Helper to extract values for 2024 vs Base 2030
def get_attr_df(row_map, label_2024, label_2030):
    records = []
    for cat, r in row_map.items():
        v_2024 = float(wd.iat[r, COL_2024])
        v_2030 = float(wd.iat[r, COL_BASE_2030])
        records.append({"Category": cat, label_2024: v_2024, label_2030: v_2030})
    return pd.DataFrame(records)

df_cap  = get_attr_df(rows_installed, "2024", "2030 (Base)")
df_pue  = get_attr_df(rows_pue,       "2024", "2030 (Base)")
df_load = get_attr_df(rows_load,      "2024", "2030 (Base)")

# --- Build dumbbell charts with subplots ---
fig = make_subplots(
    rows=3, cols=1, shared_xaxes=False, shared_yaxes=False,
    vertical_spacing=0.12,
    subplot_titles=("Installed Capacity (GW)", 
                    "Power Usage Effectiveness (PUE, lower is better)", 
                    "Load Factor (%)")
)

def add_dumbbell(row_idx, df, xaxis_title):
    # draw line segments between 2024 and 2030 for each category
    for _, row in df.iterrows():
        fig.add_trace(
            go.Scatter(
                x=[row["2024"], row["2030 (Base)"]],
                y=[row["Category"], row["Category"]],
                mode="lines",
                line=dict(width=4, color="lightgray"),
                showlegend=False,
                hoverinfo="skip"
            ),
            row=row_idx, col=1
        )
    # add markers for 2024
    fig.add_trace(
        go.Scatter(
            x=df["2024"], y=df["Category"],
            mode="markers+text",
            name="2024",
            marker=dict(size=12, color="blue"),
            text=[f"{v:.2f}" if abs(v) < 10 else f"{v:.0f}" for v in df["2024"]],
            textposition="middle left",
            hovertemplate="<b>%{y}</b><br>2024: %{x}<extra></extra>"
        ),
        row=row_idx, col=1
    )
    # add markers for 2030
    fig.add_trace(
        go.Scatter(
            x=df["2030 (Base)"], y=df["Category"],
            mode="markers+text",
            name="2030 (Base)",
            marker=dict(size=12, color="red"),
            text=[f"{v:.2f}" if abs(v) < 10 else f"{v:.0f}" for v in df["2030 (Base)"]],
            textposition="middle right",
            hovertemplate="<b>%{y}</b><br>2030: %{x}<extra></extra>"
        ),
        row=row_idx, col=1
    )
    fig.update_xaxes(title_text=xaxis_title, row=row_idx, col=1)
    fig.update_yaxes(categoryorder="array", 
                     categoryarray=["Enterprise", "Colocation & Service", "Hyperscale"], 
                     row=row_idx, col=1)

# add three panels
add_dumbbell(1, df_cap,  "GW")
add_dumbbell(2, df_pue,  "PUE (lower is better)")
add_dumbbell(3, df_load, "%")

fig.update_layout(
    title="2024 vs 2030 (Base): Hyperscale vs Colocation & Service vs Enterprise",
    height=900,
    legend_title="Period",
    margin=dict(l=80, r=40, t=80, b=40)
)

fig.show()



#### Observed that PUE has overlapping values in visualization and think it's better to represent difference in PUE to notice the little changes

In [ ]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots


file_path = "Data_annex_Energy_and_AI.xlsx"   # adjust if needed
wd = pd.read_excel(file_path, sheet_name="World Data", header=None)

# Column indices for this sheet layout
COL_2024 = 5
COL_BASE_2030 = 7

# Row indices for each attribute & category (from earlier inspection)
rows_installed = {"Hyperscale": 5, "Colocation & Service": 6, "Enterprise": 7}
rows_pue       = {"Hyperscale": 14, "Colocation & Service": 15, "Enterprise": 16}
rows_load      = {"Hyperscale": 19, "Colocation & Service": 20, "Enterprise": 21}

# Helper to extract values for 2024 vs Base 2030
def get_attr_df(row_map, label_2024="2024", label_2030="2030 (Base)"):
    recs = []
    for cat, r in row_map.items():
        v24 = float(wd.iat[r, COL_2024])
        v30 = float(wd.iat[r, COL_BASE_2030])
        recs.append({"Category": cat, label_2024: v24, label_2030: v30})
    return pd.DataFrame(recs)

df_cap  = get_attr_df(rows_installed)
df_pue  = get_attr_df(rows_pue)
df_load = get_attr_df(rows_load)

# Precompute PUE deltas (2030 - 2024)
df_pue["Δ PUE"] = df_pue["2030 (Base)"] - df_pue["2024"]

# --- Make subplots: 3 rows ---
fig = make_subplots(
    rows=3, cols=1, shared_xaxes=False, shared_yaxes=False, vertical_spacing=0.12,
    subplot_titles=("Installed Capacity (GW)",
                    "Power Usage Effectiveness (Δ from 2024 to 2030; lower is better)",
                    "Load Factor (%)")
)

# ---------- 1) Dumbbell for Capacity ----------
for _, row in df_cap.iterrows():
    fig.add_trace(go.Scatter(
        x=[row["2024"], row["2030 (Base)"]],
        y=[row["Category"], row["Category"]],
        mode="lines",
        line=dict(width=4, color="lightgray"),
        showlegend=False, hoverinfo="skip"
    ), row=1, col=1)

fig.add_trace(go.Scatter(
    x=df_cap["2024"], y=df_cap["Category"],
    mode="markers+text", name="2024",
    marker=dict(size=12, color="blue"),
    text=[f"{v:.0f}" for v in df_cap["2024"]],
    textposition="middle left",
    hovertemplate="<b>%{y}</b><br>2024: %{x}<extra></extra>"
), row=1, col=1)

fig.add_trace(go.Scatter(
    x=df_cap["2030 (Base)"], y=df_cap["Category"],
    mode="markers+text", name="2030 (Base)",
    marker=dict(size=12, color="red"),
    text=[f"{v:.0f}" for v in df_cap["2030 (Base)"]],
    textposition="middle right",
    hovertemplate="<b>%{y}</b><br>2030: %{x}<extra></extra>"
), row=1, col=1)

fig.update_xaxes(title_text="GW", row=1, col=1)
fig.update_yaxes(categoryorder="array",
                 categoryarray=["Enterprise", "Colocation & Service", "Hyperscale"],
                 row=1, col=1)

# ---------- 2) Delta bar for PUE (zero-centered) ----------
colors = df_pue["Δ PUE"].apply(lambda d: "#2ca02c" if d < 0 else "#d62728")  # green if better (down), red if worse (up)
fig.add_trace(go.Bar(
    x=df_pue["Δ PUE"], y=df_pue["Category"],
    orientation="h", marker_color=colors,
    name="Δ PUE (2030-2024)",
    text=[f"{d:+.02f}" for d in df_pue["Δ PUE"]],
    textposition="outside",
    hovertemplate="<b>%{y}</b><br>Δ PUE: %{x:+.02f}<extra></extra>"
), row=2, col=1)

# Add annotations showing 2024 → 2030 values
for _, r in df_pue.iterrows():
    fig.add_annotation(
        x=r["Δ PUE"], y=r["Category"], xref=f"x2", yref=f"y2",
        text=f"({r['2024']:.2f} → {r['2030 (Base)']:.2f})",
        showarrow=False, xanchor="left", yanchor="middle",
        font=dict(size=10)
    )

fig.update_xaxes(title_text="Δ PUE (2030 − 2024)", zeroline=True, zerolinewidth=2, zerolinecolor="#888",
                 row=2, col=1)

# ---------- 3) Dumbbell for Load Factor ----------
for _, row in df_load.iterrows():
    fig.add_trace(go.Scatter(
        x=[row["2024"], row["2030 (Base)"]],
        y=[row["Category"], row["Category"]],
        mode="lines",
        line=dict(width=4, color="lightgray"),
        showlegend=False, hoverinfo="skip"
    ), row=3, col=1)

fig.add_trace(go.Scatter(
    x=df_load["2024"], y=df_load["Category"],
    mode="markers+text", name="2024",
    marker=dict(size=12, color="blue"),
    text=[f"{v:.0f}%" for v in df_load["2024"]],
    textposition="middle left",
    hovertemplate="<b>%{y}</b><br>2024: %{x}%<extra></extra>"
), row=3, col=1)

fig.add_trace(go.Scatter(
    x=df_load["2030 (Base)"], y=df_load["Category"],
    mode="markers+text", name="2030 (Base)",
    marker=dict(size=12, color="red"),
    text=[f"{v:.0f}%" for v in df_load["2030 (Base)"]],
    textposition="middle right",
    hovertemplate="<b>%{y}</b><br>2030: %{x}%<extra></extra>"
), row=3, col=1)

fig.update_xaxes(title_text="%", row=3, col=1)
fig.update_yaxes(categoryorder="array",
                 categoryarray=["Enterprise", "Colocation & Service", "Hyperscale"],
                 row=3, col=1)

# ---------- Layout ----------
fig.update_layout(
    title="Changes from 2024 to 2030 (Base) — Hyperscale vs Colocation & Service vs Enterprise",
    height=900,
    margin=dict(l=90, r=40, t=80, b=40),
    barmode="relative",
    showlegend=True
)

# Optional: tighten PUE delta axis so tiny changes stand out
# e.g., if your deltas are ~ -0.05 to +0.08:
min_delta = min(df_pue["Δ PUE"].min(), -0.02)
max_delta = max(df_pue["Δ PUE"].max(),  0.02)
fig.update_xaxes(range=[min_delta - 0.02, max_delta + 0.02], row=2, col=1)
fig.update_layout(width=1200, height=750)
fig.show()
#fig.write_html("iea_energy_ai_dumbbell_chart.html")